In [1]:
import sqlite3
from tqdm import tqdm

In [2]:
START_YEAR = 2005
CURRENT_SEASON = 2024
SPORT_IDS = [1,11,12,13,14,15,16,17]
MONTHS = [4,5,6,7,8,9] # March folded into April, October into September

In [3]:
db = sqlite3.connect("BaseballStats.db")
cursor = db.cursor()

In [4]:
cursor.execute("DELETE FROM Level_HitterStats")
for year in tqdm(range(START_YEAR, CURRENT_SEASON + 1)):
    for level in SPORT_IDS:
        for month in MONTHS:
            
            ab, h, doubles, triples, hr, k, bb, sb, cs, hbp = cursor.execute('SELECT SUM(AB), SUM(H), SUM("2B"), SUM("3B"), SUM(HR), SUM(K), SUM(BB), SUM(SB), SUM(CS), SUM(HBP) FROM Player_Hitter_MonthStats WHERE Year=? AND Month=? AND LevelId=?', (year, month, level)).fetchone()
            
            # Some year/level combos do not have any games
            if ab == 0 or ab == None:
                continue
            
            # Calculate stats
            pa = ab + bb + hbp
            singles = h - doubles - triples - hr
            iso = (doubles + 2 * triples + 3 * hr) / ab
            avg = h / ab
            obp = (h + bb + hbp) / pa
            slg = (singles + 2 * doubles + 3 * triples + 4 * hr) / ab
            hrPerc = hr / pa
            bbPerc = bb / pa
            kPerc = k / pa
            sbRate = sb / pa
            sbPerc = sb / (sb + cs)
            # https://library.fangraphs.com/offense/woba/
            wOBA = (0.69 * bb + 0.72 * hbp + 0.89 * singles + 1.27 * doubles + 1.62 * triples + 2.10 * hr) / (pa)
            
            # Insert data
            cursor.execute("INSERT INTO Level_HitterStats VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)", (level, year, month, avg, obp, slg, iso, wOBA, hrPerc, bbPerc, kPerc, sbRate, sbPerc))
            db.commit()
            cursor = db.cursor()

100%|██████████| 20/20 [00:00<00:00, 20.73it/s]
